In [2]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
from math import isnan
import pandas as pd
from plotly_resampler import FigureResampler
import os
from datetime import datetime
import plotly.express as px
from tqdm import tqdm
import pickle
from copy import deepcopy

def every_nth(nums, nth):
    # Use list slicing to return elements starting from the (nth-1) index, with a step of 'nth'.
    return nums[nth - 1::nth]

def filter_by_time_key(df, key_name, min_time, max_time):
    return df[(df[key_name] > min_time) & (df[key_name] < max_time)]

In [ ]:

well_name = '3s-617_Run3'
engine_output_channels_path = rf'C:\Users\Jyu20\OneDrive - SLB\rhapsody\StuckPipe_investigation\code\HydraulicsCalculationPythonPackage\results_{well_name}.csv'

sample_rate = 20

if '617_Run3' in engine_output_channels_path:
    packoff_time_key = '2023-11-18T12:00:00Z'
    time_key_display_max_limit = '2023-11-18T16:00:00Z'
    time_key_display_min_limit = '2023-11-11T12:00:00Z'
elif '624_Run3' in engine_output_channels_path:
    packoff_time_key = '2024-01-02T12:00:00Z'
    time_key_display_min_limit = '2023-12-25T12:00:00Z'
    time_key_display_max_limit = '2024-01-03T12:00:00Z'
elif '624_Run1' in engine_output_channels_path:
    packoff_time_key = '2023.append('')-12-13T12:00:00Z'
    time_key_display_min_limit = '2023-12-12T12:00:00Z'
    time_key_display_max_limit = '2023-12-18T12:00:00Z'

outputs = pd.read_csv(engine_output_channels_path)
# outputs = filter_by_time_key(outputs, 'time_key',time_key_display_min_limit, time_key_display_max_limit)
outputs[['true_spps']] = outputs[['true_spps']].apply(pd.to_numeric)
outputs[['factors']] = outputs[['factors']].apply(pd.to_numeric)
outputs[['simulated_spps']] = outputs[['simulated_spps']].apply(pd.to_numeric)
print(outputs.columns)

'''
sspp, m_spp, drill_string_pressure_loss, bit depth, flowrate
'''

sub_spp_data = outputs[[
    'true_spps',
    'factors',
    'time_key',
    'simulated_spps',
    # 'flow_rate'
]].dropna()
m_spp = outputs[['true_spps', 'time_key']].dropna()

pressure_loss_max = max(
    sub_spp_data['simulated_spps'].max(),
    sub_spp_data['true_spps'].max(),
)
pressure_loss_min = min(
    sub_spp_data['simulated_spps'].min(),
    sub_spp_data['true_spps'].min(),
)

fig = go.Figure()



fig.add_trace(
    go.Scatter(
        x=sub_spp_data['time_key'], 
        y=sub_spp_data['factors'], 
        name="friction factor",
        yaxis='y'),
)

fig.add_trace(
    go.Scatter(
        x=sub_spp_data['time_key'], 
        y=sub_spp_data['simulated_spps'], 
        name="Simulated SPP",
        yaxis='y2'),
)
fig.add_trace(
    go.Scatter(
        x=m_spp['time_key'], 
        y=m_spp['true_spps'], 
        name="measured spp",
        yaxis='y3'),
)

# fig.add_trace(
#     go.Scatter(
#         x=sub_spp_data['time_key'], 
#         y=sub_spp_data['flow_rate'], 
#         name="flow rate",
#         yaxis='y4'),
# )

fig.update_layout(
    title = 'Friction factor, modeled spp and measured spp of APD2',
    yaxis=dict(title="friction factors"),
    yaxis2=dict(title="Simulated spp (pa)",
                anchor="free",
        overlaying="y",
        autoshift = True,
        range = [pressure_loss_min, pressure_loss_max]
        ),
    yaxis3=dict(title="measured spp (Pa)", anchor="free",
        overlaying="y",
        autoshift = True,
        range = [pressure_loss_min, pressure_loss_max]
        ),
    yaxis4=dict(title="flow rate", anchor="free",
        overlaying="y",
        autoshift = True,
        )
    )

fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.write_html(f"{well_name}_APD2_SPP.html")
fig.show()